# FEUP - AC
## Banking - Predicting a Loan Outcome

### Business understanding

##### Analysis of requirements with the end user
Text

##### Definition of business goals
Text

##### Translation of business goals into data mining goals
Text

### Exploratory data analysis

Let us start by importing the datasets.

In [152]:
import pandas as pd
import numpy as np

COMP_DATA_SOURCE = "comp_data/"
DEV_DATA_SOURCE = "dev_data/"

# Read in the data; merge data for tables other than loans since they do not influence merges
accounts = pd.read_csv(DEV_DATA_SOURCE + 'account.csv', sep=';')

cards_dev = pd.read_csv(DEV_DATA_SOURCE + 'card.csv', sep=';')
cards_comp = pd.read_csv(COMP_DATA_SOURCE + 'card.csv', sep=';')
cards = pd.concat([cards_dev, cards_comp])

clients = pd.read_csv(DEV_DATA_SOURCE + 'client.csv', sep=';')

dispositions = pd.read_csv(DEV_DATA_SOURCE + 'disp.csv', sep=';')

districts = pd.read_csv(DEV_DATA_SOURCE + 'district.csv', sep=';')

transactions_dev = pd.read_csv(DEV_DATA_SOURCE + 'trans.csv', sep=';')
transactions_comp = pd.read_csv(COMP_DATA_SOURCE + 'trans.csv', sep=';')
transactions = pd.concat([transactions_dev, transactions_comp])

loans_dev = pd.read_csv(DEV_DATA_SOURCE + 'loan.csv', sep=';')
loans_comp = pd.read_csv(COMP_DATA_SOURCE + 'loan.csv', sep=';')

loans_dev.head()

/tmp/ipykernel_9070/285880650.py:20: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_dev = pd.read_csv(DEV_DATA_SOURCE + 'trans.csv', sep=';')


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1


Plenty of analysis with graphs, distributions, outliers...

### Data preprocessing

Text

#### Processing accounts

The account spreadsheet relates it with the district and date in which it was created, as well with the frequency of issuance of statements with the owner.

The date comes in the from YYMMDD, which is easily extractable to the three different columns that it is aggregating. This will aid the model in detecting important patterns in the year of creation, for example, indicating a global economic crisis or prosperity when the person has created the account.

In [153]:
import src.preprocess.accounts as accpp
accounts = accpp.preprocess_accounts(accounts)
accounts.head()

,account_id,district_id,account_frequency,account_year,account_month,account_day
0,576,55,monthly issuance,1993,1,1
1,3818,74,monthly issuance,1993,1,1
2,704,55,monthly issuance,1993,1,1
3,2378,16,monthly issuance,1993,1,1
4,2632,24,monthly issuance,1993,1,2


#### Processing credit cards

In the credit cards relation, the same concept applies to the date of issuance.

MORE TEXT

In [154]:
from src.preprocess.cards import preprocess_cards

cards = preprocess_cards(cards)
cards.head()

,card_id,disp_id,type,year,month,day
0,1005,9285,classic,1993,11,7
1,104,588,classic,1994,1,19
2,747,4915,classic,1994,2,5
3,70,439,classic,1994,2,8
4,577,3687,classic,1994,2,15


#### Processing clients

In the clients relation, two foreign references and a date appear once again. However, regarding the latter one, there is an important difference: the day is the actual day plus fifty, if the client is a female, and so we can extract the gender too, here.

In [155]:
from src.preprocess.clients import preprocess_clients

clients = preprocess_clients(clients)
clients.head()

,client_id,district_id,year,month,day,gender
0,1,18,1970,12,13,female
1,2,1,1945,2,4,male
2,3,1,1940,10,9,female
3,4,5,1956,12,1,male
4,5,5,1960,7,3,female


#### Processing dispositions

The disposition describes the rights of clients to operate accounts: only "owners" can ask for loans and issue permanent orders.

There is no clear way to merge this table with loans later on, since the loans table does not refer the disposition used for the loan. Instead, let's focus on the number of dispositions and the ratio of owners of an account, which will give an impression of the nature of the people able to control the account. All humans fight and cheat on each other, or build trust together. Let's discover that pattern in Czech Republic.

In [156]:
from src.preprocess.dispositions import preprocess_dispositions
dispositions = preprocess_dispositions(dispositions)
dispositions.head()

,disp_id,client_id,account_id,type,number_account_dispositions,number_account_disponents
0,1,1,1,OWNER,1,0
1,2,2,2,OWNER,2,1
2,3,3,2,DISPONENT,2,1
3,4,4,3,OWNER,2,1
4,5,5,3,DISPONENT,2,1


#### Processing districts

The districts relation is quite dense, describing the demographic data of a region. We can extract interesting measures here:

- The commited crimes growth ratio, derived from the evolution of the number of crimes from '95 to '96
- The unemployment rate growth ratio, derived from the evolution of the employment rate from '95 to '96

We also acknowledged that there was one region where the number of crimes and unemployment rate in '95 was not known (`?` was the appearance in the respective column). In order not to discard the entire region data, we'll simply assume that number to be the same as the matching record in '96.

In [157]:
from src.preprocess.districts import preprocess_districts
districts = preprocess_districts(districts)
districts.head()

,district_id,district_no_inhabitants,district_no_cities,district_urban_inhabitants_ratio,district_average_salary,district_unemployment_rate,districts_entrepreneurs_ratio,district_crimes_per_inhabitant,district_crime_growth,district_unemploymant_growth
0,1,1204953,1,100.0,12541,0.43,0.167,0.082250,1.156752,1.482759
1,2,88884,5,46.7,8507,1.85,0.132,0.030084,1.238536,1.107784
2,3,75232,5,41.7,8980,2.21,0.111,0.037391,0.996105,1.133333
3,4,149893,6,67.4,9753,5.05,0.109,0.039308,1.123570,1.088362
4,5,95616,6,51.4,9307,4.43,0.118,0.031794,1.162080,1.150649


#### Processing loans

The loans relation references an account and the date of loan, which can be extracted. The duration seems to be in months, so let us improve the name of the column. The amounts seems to be in `Czech korunas`, the czech currency, as hinted by the list of regions; we'll convert it to euros for better visualization and understanding of the upcoming results.

In [158]:
from src.preprocess.loans import preprocess_loans
loans_comp = preprocess_loans(loans_comp)
loans_dev = preprocess_loans(loans_dev)
loans_dev.head()

,loan_id,account_id,amount,duration_months,monthly_payment,status,year,month,day
0,5314,1787,3950.66,12,329.22,-1,1993,7,5
1,5316,1801,6801.64,36,188.93,1,1993,7,11
2,6863,9188,5208.20,60,86.80,1,1993,7,28
3,5325,1843,4336.23,36,120.45,1,1993,8,3
4,7240,11013,11259.84,60,187.66,1,1993,9,6


#### Processing transactions

This table contains a lot of information that we cannot directly relate to a loan.

Let us first extract the date and make the withdrawal amounts actually negative, to give the model an impression of the direction of the money flow in and out of the bank.

For now, let us extract some statistical data related to the accounts, which we'll relate to the loans table.

In [159]:
from src.preprocess.transactions import preprocess_transactions
transactions = preprocess_transactions(transactions)
transactions.head()

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,year,month,day
0,1548749,5270,credit,credit in cash,32.79,32.79,NaN,NaN,NaN,1993,1,13
1,1548750,5270,credit,collection from another bank,1833.98,1866.76,NaN,IJ,80269753.0,1993,1,14
2,3393738,11265,credit,credit in cash,40.98,40.98,NaN,NaN,NaN,1993,1,14
3,3122924,10364,credit,credit in cash,45.08,45.08,NaN,NaN,NaN,1993,1,17
4,1121963,3834,credit,credit in cash,28.69,28.69,NaN,NaN,NaN,1993,1,19


#### Feature engineering

Text

##### Getting the average account balance based on the transactions

In [160]:
from src.feature_engineering.merge import merge_account_transactions
accounts = merge_account_transactions(accounts, transactions)
accounts.head()

,account_id,district_id,account_frequency,account_year,account_month,account_day,account_average_balance
0,576,55,monthly issuance,1993,1,1,995.213406
1,3818,74,monthly issuance,1993,1,1,1755.200383
2,704,55,monthly issuance,1993,1,1,1424.108906
3,2378,16,monthly issuance,1993,1,1,2376.927362
4,2632,24,monthly issuance,1993,1,2,1326.053913


#### Merging the data

The model will be working on the `loans` table, since the target variable belongs to it. The one foreign key of that table is `account_id`; so, to feed the model with more information about the context of the loan, we should relate concepts to the account for each table.

##### Relating cards to accounts

In [161]:
import src.feature_engineering.merge as merge
cards = merge.merge_card_account(cards, dispositions)
cards.head()

,card_id,disp_id,card_type,card_year,card_month,card_day,account_id
0,1005,9285,classic,1993,11,7,7753
1,104,588,classic,1994,1,19,489
2,747,4915,classic,1994,2,5,4078
3,70,439,classic,1994,2,8,361
4,577,3687,classic,1994,2,15,3050


##### Relating accounts with district data

Text

In [162]:
accounts = accounts.merge(districts, on="district_id").drop(axis=1, columns=["district_id"])
accounts.head()

,account_id,account_frequency,account_year,account_month,account_day,account_average_balance,district_no_inhabitants,district_no_cities,district_urban_inhabitants_ratio,district_average_salary,district_unemployment_rate,districts_entrepreneurs_ratio,district_crimes_per_inhabitant,district_crime_growth,district_unemploymant_growth
0,576,monthly issuance,1993,1,1,995.213406,157042,9,33.9,8743,2.43,0.111,0.024796,1.064225,1.292553
1,704,monthly issuance,1993,1,1,1424.108906,157042,9,33.9,8743,2.43,0.111,0.024796,1.064225,1.292553
2,192,monthly issuance,1993,1,8,1019.869554,157042,9,33.9,8743,2.43,0.111,0.024796,1.064225,1.292553
3,10364,monthly issuance,1993,1,17,1404.598889,157042,9,33.9,8743,2.43,0.111,0.024796,1.064225,1.292553
4,497,monthly issuance,1993,4,15,2244.168398,157042,9,33.9,8743,2.43,0.111,0.024796,1.064225,1.292553


##### Loading the loans table with data

In [163]:
accounts.rename(columns = {'year':'account_year', 'month':'account_month', 'day':'account_day', 'frequency': 'account_frequency', 'district_id': 'account_district_id'}, inplace=True)
loans_dev = loans_dev.merge(accounts, on="account_id", how="left")
loans_comp = loans_comp.merge(accounts, on="account_id", how="left")
loans_dev.head()

# merge with disp with card
# merge disp with account
# merge stuff

,loan_id,account_id,amount,duration_months,monthly_payment,status,year,month,day,account_frequency,...,account_average_balance,district_no_inhabitants,district_no_cities,district_urban_inhabitants_ratio,district_average_salary,district_unemployment_rate,districts_entrepreneurs_ratio,district_crimes_per_inhabitant,district_crime_growth,district_unemploymant_growth
0,5314,1787,3950.66,12,329.22,-1,1993,7,5,weekly issuance,...,502.047500,94812,10,81.8,9650,3.67,0.100,0.029574,0.939363,1.085799
1,5316,1801,6801.64,36,188.93,1,1993,7,11,monthly issuance,...,2134.584324,112709,10,73.5,8369,2.31,0.117,0.023228,0.917309,1.290503
2,6863,9188,5208.20,60,86.80,1,1993,7,28,monthly issuance,...,1232.007083,77917,5,53.5,8390,2.89,0.132,0.027234,1.020192,1.267544
3,5325,1843,4336.23,36,120.45,1,1993,8,3,monthly issuance,...,1692.519600,107870,6,58.0,8754,4.31,0.137,0.035858,1.016824,1.125326
4,7240,11013,11259.84,60,187.66,1,1993,9,6,weekly issuance,...,2343.778889,1204953,1,100.0,12541,0.43,0.167,0.082250,1.156752,1.482759


##### Replace account data with ranges

In [164]:
import src.preprocess.util as utils
loans_dev = utils.account_date_to_levels(loans_dev)
loans_comp = utils.account_date_to_levels(loans_comp)
loans_dev.head()

,loan_id,account_id,amount,duration_months,monthly_payment,status,year,month,day,account_frequency,...,district_no_inhabitants,district_no_cities,district_urban_inhabitants_ratio,district_average_salary,district_unemployment_rate,districts_entrepreneurs_ratio,district_crimes_per_inhabitant,district_crime_growth,district_unemploymant_growth,account_level
0,5314,1787,3950.66,12,329.22,-1,1993,7,5,weekly issuance,...,94812,10,81.8,9650,3.67,0.100,0.029574,0.939363,1.085799,1
1,5316,1801,6801.64,36,188.93,1,1993,7,11,monthly issuance,...,112709,10,73.5,8369,2.31,0.117,0.023228,0.917309,1.290503,2
2,6863,9188,5208.20,60,86.80,1,1993,7,28,monthly issuance,...,77917,5,53.5,8390,2.89,0.132,0.027234,1.020192,1.267544,2
3,5325,1843,4336.23,36,120.45,1,1993,8,3,monthly issuance,...,107870,6,58.0,8754,4.31,0.137,0.035858,1.016824,1.125326,3
4,7240,11013,11259.84,60,187.66,1,1993,9,6,weekly issuance,...,1204953,1,100.0,12541,0.43,0.167,0.082250,1.156752,1.482759,3


##### Remove some columns

In [165]:
loans_dev = loans_dev.drop(columns=["account_id","district_no_cities", "district_crime_growth"])
loans_comp = loans_comp.drop(columns=["account_id","district_no_cities", "district_crime_growth"])

### Training the model

Text

In [166]:
# extracting categorical values here ...
import src.preprocess.onehotencoding as encoding

for col in loans_dev.columns:
    print(col)

# one-hot encoding 
# with account_frequency
loans_dev = encoding.extract_categorical(loans_dev, "account_frequency")
loans_comp = encoding.extract_categorical(loans_comp, "account_frequency")

loan_id
amount
duration_months
monthly_payment
status
year
month
day
account_frequency
account_average_balance
district_no_inhabitants
district_urban_inhabitants_ratio
district_average_salary
district_unemployment_rate
districts_entrepreneurs_ratio
district_crimes_per_inhabitant
district_unemploymant_growth
account_level


#### Splitting the data for training

Text

In [167]:
from src.model.model import train_test_split_unbalanced
x_train, x_test, y_train, y_test = train_test_split_unbalanced(loans_dev, "status", sampling_strategy="smote", sort_by_date=False, train_ratio=0.9)

#### Feeding the model

Evaluate classification model performance with loans table

code the categorical feature into multiple binary features to allow the model to work!! (TBD)

In [168]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, roc_auc_score

# Create a Random Forest Model
random_classifier = RandomForestClassifier(n_estimators=100, random_state=0)

# Train it with data
random_classifier.fit(x_train, y_train)

# Predict the test data for dev and evaluate
predictions_dev = random_classifier.predict(x_test)
predictions_dev_prob = random_classifier.predict_proba(x_test)
print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

# Write the dev predictions to a file
predictions_dev_id = x_test["loan_id"]
predictions_dev_debt = [x[0] for x in predictions_dev_prob]
predictions_df_dev = pd.DataFrame({"Id": predictions_dev_id, "Predicted": predictions_dev_debt, "Actual": y_test})
predictions_df_dev.to_csv("output/predictions_dev.csv", index=False)

# Predict the competition data with probabilities
loans_comp_indep = loans_comp.drop(axis=1, columns=["status"])
predictions_comp_prob = random_classifier.predict_proba(loans_comp_indep).tolist()
predictions_comp = random_classifier.predict(loans_comp_indep)

# Write the comp predictions to a file
predictions_comp_debt = [x[0] for x in predictions_comp_prob]
predictions_comp_id = loans_comp["loan_id"].tolist()
predictions_df_comp = pd.DataFrame({"Id": predictions_comp_id, "Predicted": predictions_comp_debt})
predictions_df_comp.to_csv("output/predictions_comp.csv", index=False)

Recall:  0.896551724137931
Accuracy:  0.9122807017543859
Precision:  0.896551724137931
F-Score:  0.912280701754386
AUROC:  0.9125615763546798


### Another Models...

In [169]:
# Fit Naive Bayes to the training set

from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(x_train, y_train)

# Predict test set results
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.7586206896551724
Accuracy:  0.7368421052631579
Precision:  0.7407407407407407
F-Score:  0.7272727272727273
AUROC:  0.7364532019704433


In [170]:
# SVM

from sklearn.svm import SVC
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score


classifier = SVC()
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

# Check precision and recall (in this case it's better), f-measure, auc
print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.7931034482758621
Accuracy:  0.6140350877192983
Precision:  0.6666666666666666
F-Score:  0.5217391304347826
AUROC:  0.6108374384236454


In [171]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.8275862068965517
Accuracy:  0.7543859649122807
Precision:  0.7916666666666666
F-Score:  0.7307692307692307
AUROC:  0.7530788177339902


In [172]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier()
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  1.0
Accuracy:  0.5263157894736842
Precision:  1.0
F-Score:  0.0689655172413793
AUROC:  0.5178571428571429


In [173]:
from sklearn.neural_network import BernoulliRBM
lassifier = BernoulliRBM()
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.896551724137931
Accuracy:  0.7017543859649122
Precision:  0.8235294117647058
F-Score:  0.6222222222222222
AUROC:  0.6982758620689655


In [174]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.9310344827586207
Accuracy:  0.7894736842105263
Precision:  0.9
F-Score:  0.75
AUROC:  0.7869458128078818


In [175]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train, y_train)
predictions_dev = classifier.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.3103448275862069
Accuracy:  0.49122807017543857
Precision:  0.48717948717948717
F-Score:  0.5671641791044776
AUROC:  0.4944581280788177


In [176]:
classifiers = [ ("cfl1",LogisticRegression()),("cfl2",SVC()),("cfl3",GaussianNB()),("cfl4",MultinomialNB()),("cfl5",MLPClassifier())]

from sklearn.ensemble import VotingClassifier

v_c = VotingClassifier(estimators= classifiers, voting="hard",n_jobs = -1)
v_c = v_c.fit(x_train,y_train)
predictions_dev = v_c.predict(x_test)

print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
print('Accuracy: ', accuracy_score(y_test, predictions_dev))
print('Precision: ', precision_score(y_test, predictions_dev))
print('F-Score: ', f1_score(y_test, predictions_dev))
print("AUROC: ", roc_auc_score(y_test, predictions_dev))

Recall:  0.7931034482758621
Accuracy:  0.7368421052631579
Precision:  0.76
F-Score:  0.7169811320754718
AUROC:  0.7358374384236455


/home/mim/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluating the model

Text